In [1]:
from buildingmotif import BuildingMOTIF
bm = BuildingMOTIF("sqlite://") # in-memory instance
from rdflib import Namespace
from buildingmotif.dataclasses import Model

# create the namespace
BLDG = Namespace('urn:bldg/')

# Create the model! This will raise an exception if the namespace is not syntactically valid.
model = Model.create(BLDG, description="This is a test model for a simple building") 

In [2]:
# load a library
from buildingmotif.dataclasses import Library
brick = Library.load(ontology_graph="../BuildingMOTIF/libraries/brick/Brick-full.ttl")

# print the first 10 templates
print("The Brick library contains the following templates:")
for template in brick.get_templates()[:10]:
    print(f"  - {template.name}")

/Users/lazlopaul/opt/anaconda3/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 4.0.0-unsupported is an invalid version and will not be supported in a future release
  warnings.warn(
2023-01-13 12:20:01,780 | root |  WARNING: Warning: could not find dependee http://data.ashrae.org/bacnet/2020#BACnetDevice
2023-01-13 12:20:01,780 | root |  WARNING: Warning: could not find dependee https://brickschema.org/schema/Brick/ref#ifcProject


The Brick library contains the following templates:
  - https://brickschema.org/schema/Brick#Condenser_Water_Isolation_Valve
  - https://brickschema.org/schema/Brick#Supply_Air_Flow_Setpoint
  - https://brickschema.org/schema/Brick#Exhaust_Air_Flow_Proportional_Band_Parameter
  - https://brickschema.org/schema/Brick#Space_Heater
  - https://brickschema.org/schema/Brick#Building_Gas_Meter
  - https://brickschema.org/schema/Brick#Hot_Water_Differential_Pressure_Deadband_Setpoint
  - https://brickschema.org/schema/Brick#Leaving_Water_Temperature_Alarm
  - https://brickschema.org/schema/Brick#Low_Temperature_Alarm
  - https://brickschema.org/schema/Brick#Discharge_Air_Temperature_Cooling_Setpoint
  - https://brickschema.org/schema/Brick#Max_Water_Temperature_Setpoint


In [3]:
from buildingmotif.namespaces import BRICK
ahu_template = brick.get_template_by_name(BRICK.AHU)
# print template parameters
print("The template has the following parameters:")
for param in ahu_template.parameters:
    print(f"  {param}")

The template has the following parameters:
  name


In [4]:
ahu_name = "Furnace1"
ahu_binding_dict = {"name": BLDG[ahu_name]}
ahu_graph = ahu_template.evaluate(ahu_binding_dict)

# ahu_graph is just an instance of rdflib.Graph
print(ahu_graph.serialize())

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:bldg/Furnace1> a brick:AHU .




In [5]:
fan_template = brick.get_template_by_name(BRICK.Supply_Fan)
coil_template = brick.get_template_by_name(BRICK.Cooling_Coil)

# add fan
fan_name = f"{ahu_name}-Fan"
fan_binding_dict = {"name": BLDG[fan_name]}
fan_graph = fan_template.evaluate(fan_binding_dict)
model.add_graph(fan_graph)

# add clg coil
coil_name = f"{ahu_name}-Coil"
coil_binding_dict = {"name": BLDG[coil_name]}
coil_graph = coil_template.evaluate(coil_binding_dict)
model.add_graph(coil_graph)

# connecting them
model.graph.add((BLDG[ahu_name], BRICK.hasPart, BLDG[fan_name]))
model.graph.add((BLDG[ahu_name], BRICK.hasPart, BLDG[coil_name]))

# print model to confirm components were added and connected
print(model.graph.serialize())


@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

<urn:bldg/> a owl:Ontology .

<urn:bldg/Furnace1> brick:hasPart <urn:bldg/Furnace1-Coil>,
        <urn:bldg/Furnace1-Fan> .

<urn:bldg/Furnace1-Coil> a brick:Cooling_Coil .

<urn:bldg/Furnace1-Fan> a brick:Supply_Fan .




In [ ]:
# Build_model